In [20]:
%cd /mnt/storage/maxenis/coptic-machine-translation

/mnt/storage/maxenis/coptic-machine-translation


In [21]:
import pandas as pd
import re

In [22]:
dictionary = pd.read_csv('datasets/original_dictionary.csv')

In [24]:
dictionary["En"] = dictionary["En"].apply(lambda x: "|".join(re.findall(r"~~~([^;]*)", x)))

In [26]:
dictionary[dictionary["En"].isnull()]

,En,Search


In [27]:
import string

In [28]:
asci = set((string.ascii_lowercase + string.ascii_uppercase))
asci

{'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [29]:
def parse_search_field(word):
    word = str(word)
    if len(set(word) & asci) == 0: 
        word = word.split("\n")[1:-1]
        word = [w[:-1] for w in word]
        return ",".join(word)
    result = []
    word = word.split("\n")[1:-1]
    for w in word:
        if "S" in w:
            result.append(w.split("~S")[0])
    return ",".join(result)

In [30]:
dictionary["Search"] = dictionary["Search"].apply(lambda x: parse_search_field(x))

In [33]:
dictionary[dictionary["En"].str.contains("decan")]

,En,Search
8710,decan (celestial superhuman being),"ⲇⲉⲕⲁⲛⲟⲥ,ⲧⲉⲕⲁⲛⲟⲥ"
8838,decan (substantive)|decan (inverted attributive,"ⲇⲩⲕⲁⲛⲟⲥ,ⲇⲉⲕⲁⲛⲟⲥ"
11260,decan,


In [51]:
dictionary[dictionary["Search"].str.contains("ⲕⲁⲗⲉⲉⲗⲉ")]

,En,Search
1156,sonorous wooden board,"ⲕⲁⲗⲉⲗⲉ,ⲕⲁⲗⲉⲉⲗⲉ,ⲕⲁⲗⲏⲗⲉ,ⲕⲁⲗⲏⲗⲁ,ⲕⲉⲗⲉⲉⲗⲉ"


In [35]:
new_dictionary = pd.DataFrame(columns=["eng", "coptic"])

In [36]:
# iterate over rows in dictionary, split the search field and en field by comma and add a row for each pariwise combination
for index, row in dictionary.iterrows():
    if en == "" or search == "":
        continue
    en = list(set(row["En"].split("|")))
    search = list(set(row["Search"].split(",")))
    for e in en:
        for s in search:
            new_dictionary.loc[-1] = [e, s]
            new_dictionary.index = new_dictionary.index + 1
            new_dictionary = new_dictionary.sort_index()

In [37]:
# remove duplicates
new_dictionary = new_dictionary.drop_duplicates()

In [52]:
new_dictionary = new_dictionary[~(new_dictionary["coptic"] == "")]

In [54]:
new_dictionary.to_csv("datasets/raw_dictionary.csv", index=False)

In [61]:
dictionary = pd.read_csv('datasets/raw_dictionary.csv')

In [59]:
len(dictionary["coptic"].unique())

11557

In [69]:
def grammer_filter(row):
    en = str(row["eng"])
    cop = str(row["coptic"])
    if len(en.split(" ")) == 1:
        return True
    
    ratio = 2
    if len(cop) < 2:
        ratio = 4
    elif len(cop) < 4:
        ratio = 3
    return len(en) < ratio * len(cop)

In [58]:
dictionary

,eng,coptic
0,"to demonstrate (as true), i.e., prove s.th.",ϯⲕⲛⲉⲩⲉ
1,to explain s.th. to s.o.,ϯⲕⲛⲉⲩⲉ
2,"to protest, object (specific sense unclear)",ϯⲁⲙⲁⲣⲧⲉⲣⲉ
3,"to hold forth, declaim, preach",ⲁⲗⲏⲅⲓⲥⲑⲁⲓ
4,"bodily state, condition (a state of disease)",ϯⲁⲑⲩⲥⲓⲥ
...,...,...
30618,"house, region, environment",ⲁ
30619,about,ⲁ
30620,prefix of imperative,ⲁ
30621,verbal prefix perfect I,ⲁ


In [77]:
dictionary = dictionary.astype(str)

In [80]:
dictionary[dictionary["eng"].str.contains("\(|\)")]

,eng,coptic
1,object (specific sense unclear),ϯⲁⲙⲁⲣⲧⲉⲣⲉ
6,condition (a state of disease),ϯⲁⲑⲩⲥⲓⲥ
7,condition (a state of disease),ϯⲁⲑⲏⲥⲓⲥ
11,crucifixion (object: reflexive),ⲥⲧⲣⲟⲩ
12,deed),ⲉⲝⲟⲩⲥⲓⲁ
...,...,...
24118,a metal (?) object,ⲁⲗⲓⲕⲧ
24149,fenugreek (trigonella),nan
24198,advance (in age),ⲁⲓⲁⲓ
24199,increase (in seize),ⲁⲓⲁⲓ


In [71]:
dictionary[dictionary.apply(lambda x : grammer_filter(x), axis=1)]

,eng,coptic
2,to protest,ϯⲁⲙⲁⲣⲧⲉⲣⲉ
3,preach,ⲁⲗⲏⲅⲓⲥⲑⲁⲓ
4,declaim,ⲁⲗⲏⲅⲓⲥⲑⲁⲓ
5,to hold forth,ⲁⲗⲏⲅⲓⲥⲑⲁⲓ
8,bodily state,ϯⲁⲑⲩⲥⲓⲥ
...,...,...
24225,chain,ⲁⲃⲉ
24226,strap,ⲁⲃⲉ
24229,house,ⲁ
24230,about,ⲁ


,eng,coptic
0,"to demonstrate (as true), i.e., prove s.th.",ϯⲕⲛⲉⲩⲉ
1,to explain s.th. to s.o.,ϯⲕⲛⲉⲩⲉ
2,"to protest, object (specific sense unclear)",ϯⲁⲙⲁⲣⲧⲉⲣⲉ
3,"to hold forth, declaim, preach",ⲁⲗⲏⲅⲓⲥⲑⲁⲓ
4,"bodily state, condition (a state of disease)",ϯⲁⲑⲩⲥⲓⲥ
...,...,...
30618,"house, region, environment",ⲁ
30619,about,ⲁ
30620,prefix of imperative,ⲁ
30621,verbal prefix perfect I,ⲁ


In [82]:
from collections import defaultdict

In [83]:
mapping = defaultdict(list)
for index, row in dictionary.iterrows():
    en = row["eng"]
    cop = row["coptic"]
    mapping[cop] += [en]

In [214]:
def translate(word):
    split_word = word.split(" ")
    result = []
    for w in split_word:
        if len(w) == 1:
            continue
        if w in mapping:
            result.append(mapping[w])
        # else:
        #     result.append(["..."])
    return result

In [90]:
from huggingface_model import HuggingFaceTranslationModel

In [91]:
model = HuggingFaceTranslationModel.from_pretrained("models/hf/fifth_attempt-norm_romanized-finetuned-cop-eng/")

/home/bizon/anaconda3/envs/coptic/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [92]:
amir = pd.read_csv("datasets/test_data/Amir.csv")

In [93]:
# Only norm, norm_romanized and eng columns
amir = amir[["norm", "norm_romanized", "eng"]]

In [216]:
model.translate("p shhreie deie shhm a f auaksaneie auaoou a f shmshom hm peie pneieuama . neie f shoop deie peie hn n gaiaueie sha peie hooua m peief ouaoounh eievol m p iausrahl .", BEAM_GENERATION_CONFIG)

/home/bizon/anaconda3/envs/coptic/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'The child was growing, and becoming strong in spirit, and was in the desert until the day of his public appearance to Israel.'

In [100]:
amir["ml_translation"] = amir["norm_romanized"].apply(lambda x: model.translate(x, BEAM_GENERATION_CONFIG))

/home/bizon/anaconda3/envs/coptic/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://hugg

In [191]:
amir["eng"].iloc[11]

'then tell me, I will teach you what you will do to her.'

In [215]:
translate("ⲡ ϣⲏⲣⲉ ⲇⲉ ϣⲏⲙ ⲁ ϥ ⲁⲩⲝⲁⲛⲉ ⲁⲩⲱ ⲁ ϥ ϭⲙϭⲟⲙ ϩⲙ ⲡⲉ ⲡⲛⲉⲩⲙⲁ . ⲛⲉ ϥ ϣⲟⲟⲡ ⲇⲉ ⲡⲉ ϩⲛ ⲛ ϫⲁⲓⲉ ϣⲁ ⲡⲉ ϩⲟⲟⲩ ⲙ ⲡⲉϥ ⲟⲩⲱⲛϩ ⲉⲃⲟⲗ ⲙ ⲡ ⲓⲥⲣⲁⲏⲗ")

[['son, child'],
 ['and, but (specific sense unclear)',
  'and...: (marking a list or set)',
  'but, on the other hand, nonetheless (adversative)',
  'for (explanatory, where one would expect γάρ)',
  'and, so, then, for (marking shift in narrative, or change in subject',
  'especially, in particular, even (emphatic)'],
 ['sign, omen (?)',
  'small, few, young, humble',
  'small person, thing, quantity'],
 ['to grow by/through s.th.',
  'to make s.th. grow, increase s.th. in number (object: group)',
  'to make s.o. grow in strength, to strengthen s.o.',
  'to let s.o. grow, strengthen s.o. in respect to s.th.',
  'to (let) grow (specific sense or usage unclear)',
  'to grow with respect to s.th. (object: human capacity)',
  'to grow (object: none)'],
 ['or', 'and'],
 ['find strength, be able (to do)'],
 ['...'],
 ['leap, jump',
  'heaven, sky',
  'perhaps, if',
  'verbal encltic following imperf. etc.',
  'sg. masc. copula (nexus pronoun)',
  'masc. possessive article sg.'],
 ['...'],


In [128]:
amir["gpt"] = [
    "I invoke pain every lower part.",
    "Concerning the daughter and small person, hear fifty so they.",
    "They go inside, invoke each other's pity by the fleshly gap.",
    "The deceased part so they know ten by the firm so that.",
    "Give every son a cross so they go 100.",
    "Concerning this, especially she, she again here, 200.",
    "She invoked the priest, then she gave eighty in order to invoke more important ones.",
    "She summoned 200 things to send 100 by the house, truly she summoned those every year.",
    "Because if those deceased invoke, call upon, and summon, they give blessings from that temple.",
    "In the future, a priest will invoke the cross and do work, indicating debt from fifty coins.",
    "Truly, if it befall, she has a hundred coins or two hundred.",
    "Then say whether we invoke her with pity and go to her if it befall, or if it be ten, for we go and fight against the elder one, summoning them by means of a cross or a gift.",
    "You tell us every secret, whether all are hidden or not, by reason of you, do work and send, for coins, without hiding any guileful thing.",
    "Invoke the highest priest, summoning the elders, for five tongs, to raise them upward, for Satan's humbleness.",
    "Invoke the divine elder, turning outward for five nomisma coins, for help, the godly, god, to you.",
    ]

In [130]:
amir.to_csv("notebooks/amir_gpt.csv", index=False)

In [132]:
bleu = sacrebleu.metrics.BLEU()

In [133]:
max_num_translations = len(amir["eng"])

In [135]:
refs = amir["eng"].tolist()
translations_gpt = amir["gpt"].tolist()
translations_ml = amir["ml_translation"].tolist()

In [138]:
bleu.corpus_score(translations_gpt[:max_num_translations], [refs[:max_num_translations]])

BLEU = 1.20 15.2/1.5/0.4/0.2 (BP = 1.000 ratio = 1.423 hyp_len = 276 ref_len = 194)

In [139]:
bleu.corpus_score(translations_ml[:max_num_translations], [refs[:max_num_translations]])

BLEU = 19.46 50.3/23.9/14.3/8.9 (BP = 0.984 ratio = 0.985 hyp_len = 191 ref_len = 194)

In [140]:
sacrebleu.corpus_chrf(translations_gpt, [refs])

chrF2 = 19.93

In [141]:
sacrebleu.corpus_chrf(translations_ml, [refs])

chrF2 = 36.37